# TP 2

## Imports

### imports base

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import matplotlib
import plotly as py
import plotly.express as px 
import matplotlib.ticker as ticker
plt.style.use('default')
import plotly.offline as opy
import plotly.figure_factory as ff
import plotly.io as pio
from wordcloud import WordCloud
from datetime import datetime
from datetime import timedelta

import math

sns.set(style="whitegrid")

pd.options.display.float_format = '{:20,.2f}'.format 

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.metrics import log_loss

In [ ]:
!pip3 install category_encoders

     |████████████████████████████████| 81kB 3.4MB/s 


In [ ]:
import category_encoders as ce

In [ ]:
!pip3 install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp36-none-any.whl size=11685 sha256=2ee3ec096d97372943601067e7ecfc45dc2303530f452bee0d3751fc4f752d62
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built bayesian-optimization


In [ ]:
from bayes_opt import BayesianOptimization

### catboost

In [ ]:
#Ejecutar si no funciona el import
!pip3 install catboost

     |████████████████████████████████| 65.8MB 63kB/s 


In [ ]:
from catboost import CatBoostClassifier, Pool, cv

### xgboost

In [ ]:
!pip3 install xgboost

In [ ]:
import xgboost as xgb

### lightgbm

In [ ]:
!pip3 install lightgbm

In [ ]:
import lightgbm as lgb
#from lightgbm import LGBMClassifier

### ramdom forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

### perceptron

In [ ]:
	from sklearn.linear_model import Perceptron

## lectura de datos

In [ ]:
entrenamiento = pd.read_csv('Training.csv')
entrenamiento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16947 entries, 0 to 16946
Data columns (total 52 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   ID                                  16947 non-null  int64  
 1   Region                              16947 non-null  object 
 2   Territory                           16947 non-null  object 
 3   Pricing, Delivery_Terms_Quote_Appr  16947 non-null  int64  
 4   Pricing, Delivery_Terms_Approved    16947 non-null  int64  
 5   Bureaucratic_Code_0_Approval        16947 non-null  int64  
 6   Bureaucratic_Code_0_Approved        16947 non-null  int64  
 7   Submitted_for_Approval              16947 non-null  int64  
 8   Bureaucratic_Code                   16947 non-null  object 
 9   Account_Created_Date                16947 non-null  object 
 10  Source                              16947 non-null  object 
 11  Billing_Country                     16947

In [ ]:
testeo = pd.read_csv('Test.csv')
testeo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2551 entries, 0 to 2550
Data columns (total 51 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   ID                                  2551 non-null   int64  
 1   Region                              2551 non-null   object 
 2   Territory                           2551 non-null   object 
 3   Pricing, Delivery_Terms_Quote_Appr  2551 non-null   int64  
 4   Pricing, Delivery_Terms_Approved    2551 non-null   int64  
 5   Bureaucratic_Code_0_Approval        2551 non-null   int64  
 6   Bureaucratic_Code_0_Approved        2551 non-null   int64  
 7   Submitted_for_Approval              2551 non-null   int64  
 8   Bureaucratic_Code                   2551 non-null   object 
 9   Account_Created_Date                2551 non-null   object 
 10  Source                              2551 non-null   object 
 11  Billing_Country                     2551 no

In [ ]:
#hemisferios = pd.read_csv('hemisferios.csv')
#hemisferios = convertirATipo(hemisferios,['Billing_Country','hemisferio'],'category')
#hemisferios.info()

In [ ]:
#prueva de supuestos
supuestos = pd.read_csv('EvaluacionTemporal.csv',index_col='Opportunity_ID')
supuestos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1567 entries, 10689 to 12368
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Target  1567 non-null   int64
dtypes: int64(1)
memory usage: 24.5 KB


## Preprocesamiento de ambos DF

### Modificaciones de oportunidad

In [ ]:
def convertirATiempo(df,lista):
    for columna in lista:
        df[columna] = pd.to_datetime(df[columna])
    return df

def convertirATipo(df,lista,tipo):
    for columna in lista:
        df[columna] = df[columna].astype(tipo)
    return df

def convertirAFlotante(df,lista):
    for columna in lista:
        df[columna] = df[columna].apply(lambda x: None if (x == "None" or x == "Other") else float(x))
    return df

def establecerTipos(df):
    """cambia los tipos de las columnas"""
    
    #fechas
    listaDeColumnasFechas = ['Account_Created_Date','Opportunity_Created_Date','Quote_Expiry_Date','Last_Modified_Date','Delivery_Year','Planned_Delivery_End_Date','Planned_Delivery_Start_Date']
    df = convertirATiempo(df,listaDeColumnasFechas)
    #enteros
    listaDeColumnasEnteras = ['ID','Opportunity_ID']
    convertirATipo(df,listaDeColumnasEnteras,'int')
    #decimales
    listaDeColumnasEnteras = ['Price','ASP','ASP_(converted)','Total_Taxable_Amount','Total_Amount']
    convertirAFlotante(df,listaDeColumnasEnteras)
    #booleanos
    listaDeColumnasBooleanas = ['Pricing, Delivery_Terms_Quote_Appr','Pricing, Delivery_Terms_Approved','Bureaucratic_Code_0_Approval','Bureaucratic_Code_0_Approved']
    convertirATipo(df,listaDeColumnasBooleanas,'bool')
    #categoricas
    listaDeColumnasCategoricas = ['Product_Category_B','Product_Family','Region','Territory','Bureaucratic_Code','Source ','Billing_Country','Account_Name','Opportunity_Name']
    convertirATipo(df,listaDeColumnasCategoricas,'category')
    return df

In [ ]:
def agregarHemisferios(df):
  #df.reset_index()
  df = df.merge(hemisferios,how='left',on='Billing_Country')
  df['hemisferio'] = df['hemisferio'].apply(lambda x: 'Desconocido' if (x != 'Norte' and x != 'Sur') else x)
  convertirATipo(df,['hemisferio','Billing_Country'],'category')
  #df.set_index('Opportunity_ID')
  return df

In [ ]:
def eliminarColumnasSinImportancia(df):
    #"Last_Activity" y "Actual_Delivery_Date" son puros nulos
    #"Submitted_for_Approval" puros valores nulos
    #"Prod_Category_A" tiene todos sus valores iguales
    df = df.drop(['Submitted_for_Approval','Last_Activity','Actual_Delivery_Date','Prod_Category_A'], axis=1)
    return df

def agregarColumnas(df):
    df['Closed'] = df['Stage'].replace({'Closed Won': True, 'Closed Lost': True, 'Negotiation':False,'Proposal':False,'Qualification':False})#REVISAR
    df['Won'] = df['Stage'].replace({'Closed Won': True, 'Closed Lost': False, 'Negotiation':False,'Proposal':False,'Qualification':False})
    convertirATipo(df,['Stage'],'category')
    return df

def correccionDeTerritorios(df):
    #se refiere a diferentes partes de usa
    df = df.replace({'Territory':{'Central US':'USA','NE America':'USA', 'NW America':'USA','SW America':'USA','SE America':'USA'}})
    #
    df = df.replace({'Territory':{'KSA':'Saudi Arabia'}})
    #No se ha encontrado la dorma de los siguientes datos 'Kosovo','South America', 'South East Asia', 'UAE (Dubai)', 'UAE (Others)' (en el caso de kosovo no lo grafica)
    return df

def preparacionDelDataFrame(df,entrenamiento=True):
    df = eliminarColumnasSinImportancia(df)
    df = establecerTipos(df)
    df = correccionDeTerritorios(df)
    df.rename(columns = {'Source ':'Source'}, inplace = True)
    #df = agregarHemisferios(df)#desactivado desde el readcsv
    if (entrenamiento):
      df = agregarColumnas(df)
    return df

### por oportunidad

In [ ]:
def desacerseDeNulosDateTime(df,columna):
  df[columna] = df[columna].apply(lambda x: datetime.max if (x is None or x is np.NaN or x is pd.NaT) else x)
  return df

def descomposicionDeFechas(df,lista):
  """se nesesitan enteros no fechas"""
  for columna in lista:
    anio = columna + '_anio'
    df[anio] = df[columna].apply(lambda x: x if x is None else x.year)
    mes = columna + '_mes'
    df[mes] = df[columna].apply(lambda x: x if x is None else x.month)
    dia = columna + '_dia'
    df[dia] = df[columna].apply(lambda x: x if (x is None) else x.day)
  df = df.drop(lista, axis=1)
  return df

def establecerEstacion(df,columnaFecha):
  estacion = columnaFecha + '_Estacion'
  df[estacion] = 'Desconocido'
  df['temporalEstaciones'] = df[columnaFecha].apply(lambda x: 'Desconocido' if x is None else estaciones(x,True))
  df.loc[df['hemisferio'] == 'Norte' , estacion] = df['temporalEstaciones']
  df['temporalEstaciones'] = df[columnaFecha].apply(lambda x: 'Desconocido' if x is None else estaciones(x,False))
  df.loc[df['hemisferio'] == 'Sur' , estacion] = df['temporalEstaciones']
  df = df.drop(['temporalEstaciones'], axis=1)
  convertirATipo(df,[estacion],'category')
  return df

def estaciones(fecha,norte):
  if (norte):
    estaciones = ['primavera','verano','otoño','invierno']
  else:
    estaciones = ['otoño','invierno','primavera','verano']
  if (fecha.month == 12 and fecha.day >= 21 or fecha.month <= 3 and fecha.day < 21):
    return estaciones[3]
  elif (fecha.month == 9 and fecha.day >= 21 or fecha.month > 9):
    return estaciones[2]
  elif (fecha.month == 6 and fecha.day >= 21 or fecha.month > 6):
    return estaciones[1]
  elif (fecha.month == 3 and fecha.day >= 21 or fecha.month > 3):
    return estaciones[0]
  else:
    return 'Desconocido'

def diferenciaEntreFechas(df,colFechaInicial,colFechaFinal):
  df['TiempoDeDiferencia'] = (df[colFechaFinal] - df[colFechaInicial])
  df['TiempoDeDiferencia'] = pd.to_numeric(df['TiempoDeDiferencia'].dt.days, downcast='integer')#revisar si se conviete totalmente a dias
  df = df.drop([colFechaInicial,colFechaFinal], axis=1)
  return df

In [ ]:
def dataEntrenamiento(df):
  data = df.groupby('Opportunity_ID').agg({'Won':'sum','Closed':'sum',
                                           'modificaciones':'sum',
                                           #'Won':'sum','modificaciones':'sum',
                                           'Last_Modified_Date':'max','Opportunity_Created_Date':'min',
                                           #'Price':'mean',#Demaciados nulos
                                           #'Quote_Expiry_Date':'max',
                                           'Pricing, Delivery_Terms_Quote_Appr':'sum',
                                           'Pricing, Delivery_Terms_Approved':'sum',
                                           'Bureaucratic_Code_0_Approval':'sum',
                                           'Bureaucratic_Code_0_Approved':'sum',
                                           'Account_Created_Date':'min',
                                           'Total_Amount':'sum','Region':'first',
                                           #'Total_Taxable_Amount':'sum',
                                           'Territory':'first',
                                           'ASP_(converted)':'sum',
                                           'Account_Type':'first',
                                           'Delivery_Quarter':'first',
                                           'Product_Type':'first',
                                           'Product_Category_B':'first',
                                           'Billing_Country':'first',
                                           'Planned_Delivery_End_Date':'max',
                                           'Planned_Delivery_Start_Date':'min',
                                           'Delivery_Terms':'first',
                                           'Product_Family':'first',
                                           #'ASP_(converted)_Currency':'first',
                                           'TRF':'mean'
                                           #'hemisferio':'first'
                                           })#hay que seguir agregando cosas que son importantes
  return data

def dataTesteo(df):
  data = df.groupby('Opportunity_ID').agg({'modificaciones':'sum',
                                           'Last_Modified_Date':'max','Opportunity_Created_Date':'min',
                                           #'Price':'mean',#Demaciados nulos
                                           #'Quote_Expiry_Date':'max',
                                           'Pricing, Delivery_Terms_Quote_Appr':'sum',
                                           'Pricing, Delivery_Terms_Approved':'sum',
                                           'Bureaucratic_Code_0_Approval':'sum',
                                           'Bureaucratic_Code_0_Approved':'sum',
                                           'Account_Created_Date':'min',
                                           'Total_Amount':'sum','Region':'first',
                                           #'Total_Taxable_Amount':'sum',
                                           'Territory':'first',
                                           'ASP_(converted)':'sum',
                                           'Account_Type':'first',
                                           'Delivery_Quarter':'first',
                                           'Product_Type':'first',
                                           'Product_Category_B':'first',
                                           'Billing_Country':'first',
                                           'Planned_Delivery_End_Date':'max',
                                           'Planned_Delivery_Start_Date':'min',
                                           'Delivery_Terms':'first',
                                           'Product_Family':'first',
                                           #'ASP_(converted)_Currency':'first',
                                           'TRF':'mean'
                                           #'hemisferio':'first'
                                           })#hay que seguir agregando cosas que son importantes
  return data  

def agrupacionDeOportunidades(df,entrenamiento=True):
  df['modificaciones'] = 1
  if (entrenamiento):
    data = dataEntrenamiento(df)
    #Solo me interesa saber si se cerro con exito
    #data['Closed'] = data['Closed'].agg(lambda x: 1 if x > 0 else 0)
    data['Won'] = data['Won'].agg(lambda x: 1 if x > 0 else 0)
    #data['Lost'] = data['Closed'] - data['Won']#si es won es closed pero si es closed y no es won es perdido
    #data['Lost'] = data['Lost'].agg(lambda x: 1 if x > 0 else 0)

    #Los casos NO cerrados del set de entrenamiento se desconocen si tendran exito o no
    #por lo que englobarlos como triunfos o fracasos es un error deben ser retirados del set
    data = data[data['Closed'] > 0]
    data = data.drop(['Closed'], axis=1)
  else:
    data = dataTesteo(df)

  #data['Pricing, Delivery_Terms_Quote_Appr'] = data['Pricing, Delivery_Terms_Quote_Appr'].agg(lambda x: None if x == 0 else x)
  #data['Pricing, Delivery_Terms_Approved'] = data['Pricing, Delivery_Terms_Approved'].agg(lambda x: None if x == 0 else x)
  #data['Bureaucratic_Code_0_Approval'] = data['Bureaucratic_Code_0_Approval'].agg(lambda x: None if x == 0 else x)
  #data['Bureaucratic_Code_0_Approved'] = data['Bureaucratic_Code_0_Approved'].agg(lambda x: None if x == 0 else x)
  """
  data['Pricing, Delivery_Terms_Quote_Appr'] = data['Pricing, Delivery_Terms_Quote_Appr'].agg(lambda x: 1 if x > 0 else 0)
  data['Pricing, Delivery_Terms_Approved'] = data['Pricing, Delivery_Terms_Approved'].agg(lambda x: 1 if x > 0 else 0)
  data['Bureaucratic_Code_0_Approval'] = data['Bureaucratic_Code_0_Approval'].agg(lambda x: 1 if x > 0 else 0)
  data['Bureaucratic_Code_0_Approved'] = data['Bureaucratic_Code_0_Approved'].agg(lambda x: 1 if x > 0 else 0)

  #Condensando columnas
  data['Pricing, Delivery_Terms_Approved'] = data['Pricing, Delivery_Terms_Quote_Appr'] + data['Pricing, Delivery_Terms_Approved']
  data['Bureaucratic_Code_0_Approved'] = data['Bureaucratic_Code_0_Approval'] + data['Bureaucratic_Code_0_Approved']
  data = data.drop(['Bureaucratic_Code_0_Approval','Pricing, Delivery_Terms_Quote_Appr'], axis=1)

  data['Pricing, Delivery_Terms_Condensado'] = data['Pricing, Delivery_Terms_Quote_Appr'] + data['Pricing, Delivery_Terms_Approved']
  data['Pricing, Delivery_Terms_Condensado'] = data['Pricing, Delivery_Terms_Condensado'].agg(lambda x: 1 if x > 1 else 0)
  data = data.drop(['Pricing, Delivery_Terms_Quote_Appr','Pricing, Delivery_Terms_Approved'], axis=1)

  data['Bureaucratic_Code_0_Condensado'] = data['Bureaucratic_Code_0_Approval'] + data['Bureaucratic_Code_0_Approved']
  data['Bureaucratic_Code_0_Condensado'] = data['Bureaucratic_Code_0_Condensado'].agg(lambda x: 1 if x > 1 else 0)
  data = data.drop(['Bureaucratic_Code_0_Approval','Bureaucratic_Code_0_Approved'], axis=1)
  #data = data.replace({'Won':})

  data['aprobacion'] = data['Pricing, Delivery_Terms_Condensado'] + data['Bureaucratic_Code_0_Condensado']
  data['aprobacion'] = data['aprobacion'].agg(lambda x: 1 if x > 1 else 0)
  data = data.drop(['Pricing, Delivery_Terms_Condensado','Bureaucratic_Code_0_Condensado'], axis=1)

  """

  #categoricas
  #listaDeColumnasCategoricas = ['Account_Type','Region','Territory','Billing_Country',
  #                              'Delivery_Quarter','Product_Type','Product_Category_B',
  #                              'Delivery_Terms','Product_Family','ASP_(converted)_Currency']
  listaDeColumnasCategoricas = ['Account_Type','Region','Territory','Billing_Country',
                                'Delivery_Quarter','Product_Type','Product_Category_B',
                                'Delivery_Terms','Product_Family']
  convertirATipo(data,listaDeColumnasCategoricas,'category')

  #data = establecerEstacion(data,'Opportunity_Created_Date')
  #data = establecerEstacion(data,'Quote_Expiry_Date')
  #data = desacerseDeNulosDateTime(data,'Quote_Expiry_Date')#si no hay fecha de expiracion se pre supone que es infinita

  listaDeValoresDeFechas = ['Last_Modified_Date','Opportunity_Created_Date',
                            'Account_Created_Date','Planned_Delivery_Start_Date','Planned_Delivery_End_Date']
  data = descomposicionDeFechas(data,listaDeValoresDeFechas)

  #data['Diferencia_Amount'] = data['Total_Amount'] - data['Total_Amount'].mean()
  #data['Diferencia_Amount'] = data['Diferencia_Amount'].agg(lambda x: -x if x<0 else x)

  #data['Total_Amount'] = data['Total_Amount'].agg(lambda x: math.log(x,10))
  #enteros
  #listaDeColumnasEnteras = ['Quote_Expiry_Date_anio','Quote_Expiry_Date_mes','Quote_Expiry_Date_dia']
  #convertirATipo(data,listaDeColumnasEnteras,'int')
  #data = agregarHemisferios(data)

  #data = data.drop(['hemisferio'], axis=1)

  #data = diferenciaEntreFechas(data,'Planned_Delivery_Start_Date','Planned_Delivery_End_Date')

  return data

las siguientes columnas no fueron probadas hay que analizarlas

Posibles columnas a usar:

- Bureaucratic_Code
- Delivery_Year (posibles datos repetidos)
- Month (posibles datos repetidos)
- Opportunity_Owner
- Opportunity_Type
- Product_Family
- Product_Name
- Quote_Type (muy desbalanceado)

Revisar el contexto

- Total_Amount_Currency
- Total_Taxable_Amount
- Total_Taxable_Amount_Currency
- ASP
- ASP_(converted)_Currency
- ASP_Currency


las columnas con gran catidad de nones parecen afectar positivamente el resultado hay que revisarlas una por una

- Currency: Segun prueba de supuestos empeora el resultado
- Brand: Segun prueba de supuestos empeora el resultado ligeramente
- Size: Segun prueba de supuestos empeora el resultado ligeramente
- Source: Segun prueba de supuestos empeora el resultado

### categoricas a enteros (caso de que el sistema predictor no reconosca category)

In [ ]:
def convertirCategoryEnEnteros(df1,df2,columna):
  encoder = ce.BinaryEncoder(cols=[columna])
  encoder.fit(df1)
  #encoder.fit(df2)
  df1 = encoder.transform(df1)
  df2 = encoder.transform(df2)
  return df1,df2


## Ejecucion de cambios

### ejecucion por modificaciones de oportunidad

In [ ]:
entrenamientoProcesado = preparacionDelDataFrame(entrenamiento)
entrenamientoProcesado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16947 entries, 0 to 16946
Data columns (total 50 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   ID                                  16947 non-null  int64         
 1   Region                              16947 non-null  category      
 2   Territory                           16947 non-null  object        
 3   Pricing, Delivery_Terms_Quote_Appr  16947 non-null  bool          
 4   Pricing, Delivery_Terms_Approved    16947 non-null  bool          
 5   Bureaucratic_Code_0_Approval        16947 non-null  bool          
 6   Bureaucratic_Code_0_Approved        16947 non-null  bool          
 7   Bureaucratic_Code                   16947 non-null  category      
 8   Account_Created_Date                16947 non-null  datetime64[ns]
 9   Source                              16947 non-null  category      
 10  Billing_Country       

In [ ]:
testeoProsesado = preparacionDelDataFrame(testeo,False)
testeoProsesado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2551 entries, 0 to 2550
Data columns (total 47 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   ID                                  2551 non-null   int64         
 1   Region                              2551 non-null   category      
 2   Territory                           2551 non-null   object        
 3   Pricing, Delivery_Terms_Quote_Appr  2551 non-null   bool          
 4   Pricing, Delivery_Terms_Approved    2551 non-null   bool          
 5   Bureaucratic_Code_0_Approval        2551 non-null   bool          
 6   Bureaucratic_Code_0_Approved        2551 non-null   bool          
 7   Bureaucratic_Code                   2551 non-null   category      
 8   Account_Created_Date                2551 non-null   datetime64[ns]
 9   Source                              2551 non-null   category      
 10  Billing_Country         

### ejecucion por oportunidad

In [ ]:
agrupacionEntrenamiento = agrupacionDeOportunidades(entrenamientoProcesado)
agrupacionEntrenamiento.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9791 entries, 0 to 12803
Data columns (total 33 columns):
 #   Column                              Non-Null Count  Dtype   
---  ------                              --------------  -----   
 0   Won                                 9791 non-null   int64   
 1   modificaciones                      9791 non-null   int64   
 2   Pricing, Delivery_Terms_Quote_Appr  9791 non-null   int64   
 3   Pricing, Delivery_Terms_Approved    9791 non-null   int64   
 4   Bureaucratic_Code_0_Approval        9791 non-null   int64   
 5   Bureaucratic_Code_0_Approved        9791 non-null   int64   
 6   Total_Amount                        9791 non-null   float64 
 7   Region                              9791 non-null   category
 8   Territory                           9791 non-null   category
 9   ASP_(converted)                     9791 non-null   float64 
 10  Account_Type                        9791 non-null   category
 11  Delivery_Quarter             

In [ ]:
agrupacionTesteo = agrupacionDeOportunidades(testeoProsesado,False)
agrupacionTesteo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1567 entries, 10689 to 12368
Data columns (total 32 columns):
 #   Column                              Non-Null Count  Dtype   
---  ------                              --------------  -----   
 0   modificaciones                      1567 non-null   int64   
 1   Pricing, Delivery_Terms_Quote_Appr  1567 non-null   int64   
 2   Pricing, Delivery_Terms_Approved    1567 non-null   int64   
 3   Bureaucratic_Code_0_Approval        1567 non-null   int64   
 4   Bureaucratic_Code_0_Approved        1567 non-null   int64   
 5   Total_Amount                        1567 non-null   float64 
 6   Region                              1567 non-null   category
 7   Territory                           1567 non-null   category
 8   ASP_(converted)                     1567 non-null   float64 
 9   Account_Type                        1567 non-null   category
 10  Delivery_Quarter                    1567 non-null   category
 11  Product_Type             

### separacion de resultados de entrenamiento

In [ ]:
entrenamientoResultado = agrupacionEntrenamiento[['Won']]
entrenamientoResultado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9791 entries, 0 to 12803
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Won     9791 non-null   int64
dtypes: int64(1)
memory usage: 153.0 KB


In [ ]:
agrupacionEntrenamiento = agrupacionEntrenamiento.drop(['Won'], axis=1)
agrupacionEntrenamiento.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9791 entries, 0 to 12803
Data columns (total 32 columns):
 #   Column                              Non-Null Count  Dtype   
---  ------                              --------------  -----   
 0   modificaciones                      9791 non-null   int64   
 1   Pricing, Delivery_Terms_Quote_Appr  9791 non-null   int64   
 2   Pricing, Delivery_Terms_Approved    9791 non-null   int64   
 3   Bureaucratic_Code_0_Approval        9791 non-null   int64   
 4   Bureaucratic_Code_0_Approved        9791 non-null   int64   
 5   Total_Amount                        9791 non-null   float64 
 6   Region                              9791 non-null   category
 7   Territory                           9791 non-null   category
 8   ASP_(converted)                     9791 non-null   float64 
 9   Account_Type                        9791 non-null   category
 10  Delivery_Quarter                    9791 non-null   category
 11  Product_Type                 

## Prueva

### prueba supuestamete bien (arbitraria con un error aprox de +0.04 en logloss)

In [ ]:
def pruebaArbitraria():
  serieDiferenciasDeSupuestos = dataResultante['Target'] - supuestos['Target']
  serieDiferenciasDeSupuestos = serieDiferenciasDeSupuestos.agg(lambda x: abs(x))
  #valores a vencer
  #0.29328282239325076 0.066288348418729 0.25746523730152193
  #0.31706383931591736 0.05312227841294053 0.23048270740543753 ramdom forest
  #0.289665541459807 0.07066249882454302 0.26582418780942985 catboost
  #0.29650158086984807 0.06760957650563981 0.2600184157048108 //0.42
  print(serieDiferenciasDeSupuestos.mean(),serieDiferenciasDeSupuestos.var(),serieDiferenciasDeSupuestos.std())
  log_loss(supuestos['Target'],dataResultante['Target'])#predijo 0.4774430257846605 mientras el resultado dio 0.43117
#pruebaArbitraria()

0.29554548477095083 0.06262130712858385 0.2502424966479192


0.4628022823299211

In [ ]:
#dataResultante['Target'].agg(lambda x: 1 if x < 0.55 and x > 0.45 else 0).value_counts()

In [ ]:
#dataResultante['Target'].agg(lambda x: 1 if x > 0.95 or x < 0.05 else 0).value_counts()

In [ ]:
def pruebaTestDeEliminacion():
  #Prueba de eliminacion de columnas
  listaDeCategoricas = ['Region','Territory','Product_Type','Product_Category_B',
                        'Account_Type','Delivery_Quarter','Billing_Country',
                        'Delivery_Terms','Product_Family']

  maxData = pd.DataFrame()
  maxValor = -9999
  for col in agrupacionEntrenamiento.columns:
    desacerse = [col]
    agrupacionEntrenamientoCat = agrupacionEntrenamiento.drop(desacerse,axis=1)
    agrupacionTesteoCat = agrupacionTesteo.drop(desacerse,axis=1)
    print(col)

    llistaAux = []
    for elemento in listaDeCategoricas:
      if (elemento in desacerse):
        continue
      llistaAux.append(elemento)
    #listaDeCategoricas = []
    res,importancia = catboost(agrupacionEntrenamientoCat,entrenamientoResultado,llistaAux,agrupacionTesteoCat)
    valor = testeoRes(agrupacionTesteoCat,res)
    if (valor > maxValor):
      print("--------------")
      print(valor)
      print("--------------")
      maxValor = valor
      maxData = res
  print(maxValor)
  #0.4628022823299211 cat
  return maxData

## Resultado

In [ ]:
def generarResultado(df,prediccion):
  resultado = df.copy(True)
  resultado['Target'] = prediccion
  resultado = resultado[['Target']]
  resultado.to_csv('resultado.csv')
  return resultado

In [ ]:
def auxiliarUnificarValor(arreglo):
  return arreglo[:,0]

## Catboost

### grid search

In [ ]:
def GridSearchCatboost(train_data_o,train_labels,cat_features):
  """clf = CatBoostClassifier()
  params = {'iterations': [500],
            'depth': [4, 5, 6],
            #'loss_function': ['Logloss', 'CrossEntropy'],
            'l2_leaf_reg': np.logspace(-10, 1, 50),
            'leaf_estimation_iterations': [10],
            'learning_rate': [0.5,0.25,0.05],
            'rsm' : [0.7,0.5]
            #'eval_metric': ['Accuracy'],
            #'use_best_model': ['True'],
            #'logging_level':['Silent'],
            #'random_seed': [42]
          }
  scorer = make_scorer(accuracy_score)
  clf_grid = GridSearchCV(estimator=clf, param_grid=params, scoring=scorer, cv=5)
  clf_grid.fit(X_train_enc, y_train,cat_features)
  best_param = clf_grid.best_params_
  return best_param"""
  model = CatBoostClassifier(loss_function='Logloss')
  #model = CatBoostClassifier(loss_function='CrossEntropy')

  train_data = train_data_o
  # convert categorical columns to integers
  for header in cat_features:
    train_data[header] = train_data[header].astype('category').cat.codes

  grid = {'learning_rate': list(np.arange(0.05,0.51,0.05)),
          'depth': list(range(6,13,2)),
          'l2_leaf_reg': list(np.arange(2,5.1,0.5)),
          'iterations': list(range(10,101,15)),
          'rsm':list(np.arange(0.1,1.1,0.1))
          }

  grid_search_result = model.grid_search(grid, 
                                        X=train_data, 
                                        y=train_labels,
                                        cv=10,
                                        plot=True)
  
  return grid_search_result

In [ ]:
listaDeCategoricas = ['Region','Territory','Product_Type','Product_Category_B',
                      'Account_Type','Delivery_Quarter','Billing_Country']
#comentado para evitar ejecucion erronea
#valores = GridSearchCatboost(agrupacionEntrenamiento,entrenamientoResultado,listaDeCategoricas)
#valores

### bayesian

In [ ]:
def catboost_classifier(learning_rate, depth, l2_leaf_reg,iterations,rsm):
    params = {
        "loss_function": "Logloss",
        #"eval_metric" : "AUC", 
        "depth" : int(depth),
        "l2_leaf_reg" : l2_leaf_reg,
        "learning_rate" : learning_rate,
        #"random_state" : 42,
        "rsm" : rsm,
        "iterations": int(iterations)
        #"logging_level" : "Silent",
        #"thread_count": 24,
        #"num_boost_round": int(num_boost_round)
    }
    X_train = agrupacionEntrenamiento
    y_train = entrenamientoResultado
    categorical_features = ['Region','Territory','Product_Type','Product_Category_B',
                            'Account_Type','Delivery_Quarter','Billing_Country']

    train_data = Pool(data=X_train, label=y_train, 
                               cat_features=categorical_features)
    
    cv_result = cv(train_data,
                       params,
                       early_stopping_rounds=100,
                       stratified=True,
                       nfold=3)
    return cv_result['train-Logloss-mean'].iloc[-1]

def bayesianoCatboost():
  grid = {'learning_rate': (0.01,0.10),
          'depth': (8,14),
          'l2_leaf_reg': (0.1,3.0),
          'iterations': (10,100),
          'rsm':(0.01,1.0)
          }
  catboostBO = BayesianOptimization(catboost_classifier, grid)

  return catboostBO.maximize(n_iter=15, init_points=2)

In [ ]:
#bayesianoCatboost()
#catboost_classifier(0.05,10,3.5,20,0.5)

### prediccion

In [ ]:
def catboost(train_data,train_labels,cat_features,eval_data):
  # Inicializador de CatBoostClassifier
  model = CatBoostClassifier(iterations=100,
                             learning_rate=0.4,#0.5
                             depth=7,#8
                             l2_leaf_reg=4,#3
                             rsm=0.7,#0.5
                             #random_strength=0.75,
                             grow_policy='SymmetricTree'#'SymmetricTree', 'Depthwise' y 'Lossguide'
                             #random_seed=2
                             #use_best_model=True,#Requiere data de evalucion
                             #eval_set=supuestos
                             )
  #parametros importantes por agregar: l2_leaf_reg, rsm
  #para tipos de category: cat_features, one_hot_max_size

  # Entrenamiento del modelo
  #model.fit(train_data, train_labels, cat_features)

  eval_dataset = Pool(data=eval_data,
                      label=supuestos[['Target']],
                      cat_features=cat_features)
  model.fit(train_data, train_labels, cat_features, use_best_model=True, eval_set=eval_dataset)

  # obtener prediccion
  #preds_class = model.predict(eval_data)

  # obtener probabilidad de prediccion
  preds_proba = model.predict_proba(eval_data)

  # obtener RawFormulaVal de la prediccion
  #preds_raw = model.predict(eval_data, prediction_type='RawFormulaVal')

  #return resultado(eval_data,preds_proba)
  importancia = model.get_feature_importance(type= "FeatureImportance",prettified=True)
  return preds_proba,importancia

In [ ]:
listaDeCategoricas = ['Region','Territory','Product_Type','Product_Category_B',
                      'Account_Type','Delivery_Quarter','Billing_Country',
                      'Delivery_Terms','Product_Family']
res,importancia = catboost(agrupacionEntrenamiento.copy(deep=True),entrenamientoResultado.copy(deep=True),listaDeCategoricas,agrupacionTesteo.copy(deep=True))
res

In [ ]:
resultado = auxiliarUnificarValor(res)
#resultado = res
resultado = (resultado - 1) * (-1)#En las ultimas pruevas es el opuesto el que genera mejor resultado
resultado

array([0.52206345, 0.42744145, 0.60402653, ..., 0.20138953, 0.3453228 ,
       0.26537745])

In [ ]:
dataResultante = generarResultado(agrupacionTesteo,resultado)
dataResultante.head()

,Target
Opportunity_ID,
10689,0.52
10690,0.43
10691,0.60
10692,0.45
10693,0.77


In [ ]:
importancia

,Feature Id,Importances
0,Product_Type,19.21
1,TRF,8.59
2,Region,6.57
3,Planned_Delivery_Start_Date_dia,5.29
4,Account_Created_Date_anio,4.86
5,Account_Type,4.81
6,Planned_Delivery_Start_Date_mes,4.43
7,Opportunity_Created_Date_mes,4.29
8,Opportunity_Created_Date_anio,3.68
9,"Pricing, Delivery_Terms_Approved",3.62


## XGBoost

In [ ]:
def xgboost(train_set, train_label,listaCategorica,test_set,masImportantes=0):
  xgb_reg = xgb.XGBClassifier(max_depth = 8, 
                           colsample_bytree = 0.5, 
                           learning_rate = 0.3, 
                           alpha = 8,
                           min_child_weight = 0.5,
                           n_estimators = 200,
                           objetive = 'reg:squarederror')
  xgb_reg.fit(train_set, train_label)
  predicts = xgb_reg.predict_proba(test_set)
  if (masImportantes > 0):
    xgb.plot_importance(xg_reg, max_num_features=masImportantes)
  return predicts




In [ ]:
#listaDeCategoricas = ['Region','Territory','Product_Type','Product_Category_B']
#listaDeCategoricas = ['Region','Territory','Product_Type','Product_Category_B','Account_Type','Delivery_Quarter']
listaDeCategoricas = ['Region','Territory','Product_Type','Product_Category_B',
                      'Account_Type','Delivery_Quarter','Billing_Country',
                      'Delivery_Terms','Product_Family','ASP_(converted)_Currency']
#listaDeCategoricas = []
agrupacionEntrenamientoXGB = agrupacionEntrenamiento.copy(True)
agrupacionTesteoXGB = agrupacionTesteo.copy(True)

# convert categorical columns to integers 
#usando la funcion de catboost
for header in listaDeCategoricas:
  agrupacionEntrenamientoXGB[header] = agrupacionEntrenamientoXGB[header].astype('category').cat.codes
  agrupacionTesteoXGB[header] = agrupacionTesteoXGB[header].astype('category').cat.codes
#usando metodo personalizado
#for columna in listaDeCategoricas:
#  agrupacionEntrenamientoXGB,agrupacionTesteoXGB = convertirCategoryEnEnteros(agrupacionEntrenamientoXGB,agrupacionTesteoXGB,columna)


agrupacionEntrenamientoXGB.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9791 entries, 0 to 12803
Data columns (total 34 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   modificaciones                      9791 non-null   int64  
 1   Pricing, Delivery_Terms_Quote_Appr  9791 non-null   int64  
 2   Pricing, Delivery_Terms_Approved    9791 non-null   int64  
 3   Bureaucratic_Code_0_Approval        9791 non-null   int64  
 4   Bureaucratic_Code_0_Approved        9791 non-null   int64  
 5   Total_Amount                        9791 non-null   float64
 6   Region                              9791 non-null   int8   
 7   Total_Taxable_Amount                9791 non-null   float64
 8   Territory                           9791 non-null   int8   
 9   ASP_(converted)                     9791 non-null   float64
 10  Account_Type                        9791 non-null   int8   
 11  Delivery_Quarter                    9791 n

In [ ]:

agrupacionTesteoXGB.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1567 entries, 10689 to 12368
Data columns (total 34 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   modificaciones                      1567 non-null   int64  
 1   Pricing, Delivery_Terms_Quote_Appr  1567 non-null   int64  
 2   Pricing, Delivery_Terms_Approved    1567 non-null   int64  
 3   Bureaucratic_Code_0_Approval        1567 non-null   int64  
 4   Bureaucratic_Code_0_Approved        1567 non-null   int64  
 5   Total_Amount                        1567 non-null   float64
 6   Region                              1567 non-null   int8   
 7   Total_Taxable_Amount                1567 non-null   float64
 8   Territory                           1567 non-null   int8   
 9   ASP_(converted)                     1567 non-null   float64
 10  Account_Type                        1567 non-null   int8   
 11  Delivery_Quarter                    15

In [ ]:

res = xgboost(agrupacionEntrenamientoXGB,entrenamientoResultado,listaDeCategoricas,agrupacionTesteoXGB)
res

array([[0.2085638 , 0.7914362 ],
       [0.73569804, 0.26430196],
       [0.08843356, 0.91156644],
       ...,
       [0.990908  , 0.00909197],
       [0.00688678, 0.9931132 ],
       [0.9187886 , 0.0812114 ]], dtype=float32)

In [ ]:
resultado = auxiliarUnificarValor(res)
resultado = (resultado - 1) * (-1)#En las ultimas pruevas es el opuesto el que genera mejor resultado
resultado

array([0.7914362 , 0.26430196, 0.91156644, ..., 0.00909197, 0.9931132 ,
       0.08121139], dtype=float32)

In [ ]:
dataResultante = generarResultado(agrupacionTesteo,resultado)
dataResultante.head()

,Target
Opportunity_ID,
10689,0.79
10690,0.26
10691,0.91
10692,0.51
10693,1.00


## LightGBM

In [ ]:
def lightGBM(train_data,train_labels,cat_features,eval_data):
  """# evaluate the model
  model = lgb.LGBMClassifier()
  cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
  n_scores = cross_val_score(model,
                             train_data,
                             train_labels,
                             scoring='accuracy',
                             cv=cv,
                             n_jobs=-1,
                             categorical_feature=cat_features,
                             error_score='raise')
  print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
  # fit the model on the whole dataset
  model.fit(X, y)
  # make a single prediction
  yhat = model.predict(eval_data)
  return yhat"""

  #
  # Create training and validation sets
  #
  x, x_test, y, y_test = train_test_split(train_data,train_labels, test_size=0.2, random_state=42, stratify=train_labels)

  #
  # Create the LightGBM data containers
  #
  #categorical_features = [c for c, col in enumerate(train.columns) if 'cat' in col]
  categorical_features = cat_features
  train_data = lgb.Dataset(x, label=y, categorical_feature=categorical_features)
  test_data = lgb.Dataset(x_test, label=y_test)


  #
  # Train the model
  #

  parameters = {
      'application': 'binary',
      'objective': 'binary',
      'metric': 'auc',
      'is_unbalance': 'true',
      'boosting': 'gbdt',
      'num_leaves': 31,
      'feature_fraction': 0.5,
      'bagging_fraction': 0.5,
      'bagging_freq': 20,
      'learning_rate': 0.05,
      'verbose': 0
  }

  model = lgb.train(parameters,
                    train_data,
                    valid_sets=test_data,
                    num_boost_round=5000,
                    early_stopping_rounds=100)


  submission = eval_data
  ids = submission.index.values
  #submission.drop('id', inplace=True, axis=1)


  x = submission.values
  y = model.predict(x)

  output = pd.DataFrame({'id': ids, 'target': y})
  output.to_csv("submission.csv", index=False)

  prediccion = model.predict(y)

  #ids = eval_data['Opportunity_ID'].values
  #output = pd.DataFrame({'Opportunity_ID': ids, 'target': y})
  return prediccion

In [ ]:
listaDeCategoricas = ['Region','Territory','Product_Type','Product_Category_B','Account_Type','Delivery_Quarter','Billing_Country']
res = lightGBM(agrupacionEntrenamiento,entrenamientoResultado,listaDeCategoricas,agrupacionTesteo)
res

[1]	valid_0's auc: 0.79851
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.863986
[3]	valid_0's auc: 0.869374
[4]	valid_0's auc: 0.877967
[5]	valid_0's auc: 0.880718
[6]	valid_0's auc: 0.88335
[7]	valid_0's auc: 0.885716
[8]	valid_0's auc: 0.882643
[9]	valid_0's auc: 0.882596
[10]	valid_0's auc: 0.884059
[11]	valid_0's auc: 0.883799
[12]	valid_0's auc: 0.884385
[13]	valid_0's auc: 0.886554
[14]	valid_0's auc: 0.885845
[15]	valid_0's auc: 0.885445
[16]	valid_0's auc: 0.886275
[17]	valid_0's auc: 0.885326
[18]	valid_0's auc: 0.886202
[19]	valid_0's auc: 0.887516
[20]	valid_0's auc: 0.888453
[21]	valid_0's auc: 0.889643
[22]	valid_0's auc: 0.890854
[23]	valid_0's auc: 0.891974
[24]	valid_0's auc: 0.892554
[25]	valid_0's auc: 0.893594
[26]	valid_0's auc: 0.893797
[27]	valid_0's auc: 0.894586
[28]	valid_0's auc: 0.896922
[29]	valid_0's auc: 0.898042
[30]	valid_0's auc: 0.898728
[31]	valid_0's auc: 0.899502
[32]	valid_0's auc: 0.90003
[33]	valid_0's auc: 0

ValueError: ignored

In [ ]:
resultado = pd.Series(res)

#ids = eval_data['Opportunity_ID'].values
#resultado = pd.DataFrame({'Opportunity_ID': ids, 'target': resultado})
#resultado = (resultado - 1) * (-1)#En las ultimas pruevas es el opuesto el que genera mejor resultado
resultado

0                      0.95
1                      0.21
2                      0.92
3                      0.80
4                      0.02
               ...         
7827                   0.02
7828                   0.90
7829                   1.00
7830                   0.06
7831                   0.00
Length: 7832, dtype: float64

In [ ]:
dataResultante = generarResultado(agrupacionTesteo,resultado)
dataResultante.head()

,Target
Opportunity_ID,
10689,nan
10690,nan
10691,nan
10692,nan
10693,nan


## Ramdom forest

### bayesiano

In [ ]:
#Bayesian optimization
def bayesian_optimization(dataset, function, parameters):
   X_train, y_train, X_test, y_test = dataset
   n_iterations = 10
   gp_params = {"alpha": 1e-4}

   BO = BayesianOptimization(function, parameters)
   BO.maximize(n_iter=n_iterations, **gp_params)

   return BO.max

def rfc_optimization(cv_splits):
    def function(n_estimators, oob_score, min_samples_split):
        desacerse = ['Planned_Delivery_End_Date_anio','Planned_Delivery_End_Date_mes','Planned_Delivery_End_Date_dia','Last_Modified_Date_dia']
        X_train = agrupacionEntrenamientoXGB.drop(desacerse,axis=1)
        y_train = entrenamientoResultado
        return cross_val_score(
               RandomForestClassifier(
                   n_estimators=int(max(n_estimators,0)),
                   #max_depth=int(max(max_depth,1)),
                   min_samples_split=int(max(min_samples_split,2)),
                   n_jobs=-1,
                   #random_state=42,
                   oob_score=(max(oob_score,1)),
                   bootstrap=True,
                   #class_weight="balanced"
                   max_features='sqrt'
                  ),
               X=X_train, 
               y=y_train, 
               cv=cv_splits,
               scoring="roc_auc",
               n_jobs=-1).mean()

    parameters = {"n_estimators": (200, 1000),
                  "oob_score": (5, 30),
                  "min_samples_split": (2, 10)}
    
    return function, parameters

#Train model
def ramdomForestBayesiano(X_train, y_train, X_test, y_test):
    dataset = (X_train, y_train, X_test, y_test)
    cv_splits = 4

    function, parameters = rfc_optimization(cv_splits)
    
    best_solution = bayesian_optimization(dataset, function, parameters)      
    params = best_solution["params"]

    model = RandomForestClassifier(
             n_estimators=int(max(params["n_estimators"], 0)),
             #max_depth=int(max(params["max_depth"], 1)),
             oob_score=(max('oob_score',1)),
             min_samples_split=int(max(params["min_samples_split"], 2)), 
             n_jobs=-1, 
             random_state=42,   
             class_weight="balanced")

    model.fit(X_train, y_train)
    predictions = model.predict_proba(X_test)
    return predictions

In [ ]:
#Uso los mismos valores de XGB para las variables categoricas ya que no las soporta
desacerse = ['Planned_Delivery_End_Date_anio','Planned_Delivery_End_Date_mes','Planned_Delivery_End_Date_dia','Last_Modified_Date_dia']
#comentado para evitar ejecucion erronea
#res = ramdomForestBayesiano(agrupacionEntrenamientoXGB.drop(desacerse,axis=1),
#                            entrenamientoResultado,
#                            agrupacionTesteoXGB.drop(desacerse,axis=1),
#                            supuestos[['TargetBinario']])
#res

|   iter    |  target   | min_sa... | n_esti... | oob_score |
-------------------------------------------------------------
|  1        |  0.8754   |  5.516    |  919.3    |  16.95    |
|  2        |  0.8742   |  5.209    |  219.3    |  13.67    |
|  3        |  0.8752   |  4.422    |  555.0    |  16.9     |
|  4        |  0.8752   |  8.854    |  393.1    |  24.81    |
|  5        |  0.8755   |  6.257    |  571.6    |  14.75    |
|  6        |  0.8748   |  8.591    |  1e+03    |  7.999    |
|  7        |  0.8743   |  7.239    |  200.1    |  29.67    |
|  8        |  0.8747   |  6.585    |  1e+03    |  25.98    |
|  9        |  0.8753   |  4.613    |  999.8    |  29.7     |
|  10       |  0.8729   |  7.116    |  200.1    |  29.46    |
|  11       |  0.8752   |  3.324    |  999.8    |  14.0     |
|  12       |  0.8735   |  7.286    |  200.1    |  5.075    |
|  13       |  0.8751   |  8.459    |  999.9    |  26.34    |
|  14       |  0.8732   |  6.593    |  200.1    |  9.143    |
|  15   

TypeError: ignored

### prediccion

In [ ]:
def ramdomForest(train_data,train_labels,eval_data):
  # Crear el modelo
  #Si no se le coloca un valor de max depth lo toma por min_samples_leaf a la creacion de hojas
  model = RandomForestClassifier(n_estimators=1000,
                                 #max_features=5,
                                 min_samples_leaf=9,
                                 oob_score=9,#7
                                 bootstrap=True,
                                 verbose=0,
                                 #max_leaf_nodes=20,
                                 #min_weight_fraction_leaf=0.01,
                                 max_features=7#7
                                 )
  #parametros importantes
  #  n_estimators: será la cantidad de árboles que generaremos.
  #  max_features: la manera de seleccionar la cantidad máxima de features para cada árbol.
  #  min_sample_leaf: número mínimo de elementos en las hojas para permitir un nuevo split (división) del nodo.
  #  oob_score: es un método que emula el cross-validation en árboles y permite mejorar la precisión y evitar overfitting.
  #  boostrap: para utilizar diversos tamaños de muestras para entrenar. Si se pone en falso, utilizará siempre el dataset completo.
  #  n_jobs: si tienes multiples cores en tu CPU, puedes indicar cuantos puede usar el modelo al entrenar para acelerar el entrenamiento.
  # a entrenar!
  model.fit(train_data, train_labels)
  predictions = model.predict_proba(eval_data)
  return predictions
  

In [ ]:
#Uso los mismos valores de XGB para las variables categoricas ya que no las soporta
desacerse = ['Planned_Delivery_End_Date_anio','Planned_Delivery_End_Date_mes','Planned_Delivery_End_Date_dia','Last_Modified_Date_dia','Total_Taxable_Amount','ASP_(converted)_Currency']
agrupacionEntrenamientoRF = agrupacionEntrenamientoXGB.drop(desacerse,axis=1)
agrupacionTesteoRF = agrupacionTesteoXGB.drop(desacerse,axis=1)
#agrupacionTesteoRF = agrupacionTesteoXGB.drop(['Target'],axis=1)
print(len(agrupacionEntrenamientoRF.columns) - len(agrupacionTesteoRF.columns))

0


In [ ]:
res = ramdomForest(agrupacionEntrenamientoRF,
                   entrenamientoResultado,
                   agrupacionTesteoRF)
res

array([[0.27503331, 0.72496669],
       [0.39253994, 0.60746006],
       [0.40023432, 0.59976568],
       ...,
       [0.54602135, 0.45397865],
       [0.42050524, 0.57949476],
       [0.71256341, 0.28743659]])

In [ ]:
resultado = auxiliarUnificarValor(res)
resultado = (resultado - 1) * (-1)#En las ultimas pruevas es el opuesto el que genera mejor resultado
resultado

array([0.72496669, 0.60746006, 0.59976568, ..., 0.45397865, 0.57949476,
       0.28743659])

In [ ]:
dataResultante = generarResultado(agrupacionTesteo,resultado)
dataResultante.head()

,Target
Opportunity_ID,
10689,0.72
10690,0.61
10691,0.60
10692,0.47
10693,0.77


## Perceptron

In [ ]:
for header in cat_features:
    agrupacionEntrenamiento[header] = agrupacionEntrenamiento[header].astype('category').cat.codes

In [ ]:
def perceptron(train_set, train_label, listaDeCategoricas, test_set):
  perceptron_mod = Perceptron(tol=1e-3, random_state=0)
  for header in listaDeCategoricas:
    train_set[header] = train_set[header].astype('category').cat.codes
  
  for header in listaDeCategoricas:
    test_set[header] = test_set[header].astype('category').cat.codes
  perceptron_mod.fit(train_set, train_label)
    
  predicts = perceptron_mod.predict(test_set)
  return predicts


In [ ]:
desacerse = ['Planned_Delivery_End_Date_anio','Planned_Delivery_End_Date_mes','Planned_Delivery_End_Date_dia','Last_Modified_Date_dia']
agrupacionEntrenamientoPerceptron = agrupacionEntrenamiento.drop(desacerse,axis=1)
agrupacionTesteoPerceptron = agrupacionTesteo.drop(desacerse,axis=1)

In [ ]:
listaDeCategoricas = ['Region',
                      'Territory', 
                      'Product_Family', 
                      'Delivery_Terms', 
                      'Product_Type',
                      'Product_Category_B',
                      'Account_Type',
                      'Delivery_Quarter',
                      'Billing_Country',
                      'ASP_(converted)_Currency']
resPerceptron = perceptron(agrupacionEntrenamientoPerceptron, entrenamientoResultado, listaDeCategoricas, agrupacionTesteoPerceptron)
resPerceptron

array([0, 0, 1, ..., 1, 1, 1])

In [ ]:
dataResultante = generarResultado(agrupacionTesteo,resPerceptron)
dataResultante.head()

,Target
Opportunity_ID,
10689,0
10690,0
10691,1
10692,0
10693,1
